In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
sys.path.append(parent_folder)

In [2]:
import optuna
import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.MLP import MLP
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 1001}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_MLP = {'layer_dims': None, 'activations': 'tanh', 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 1, 10
D_list = D = np.linspace(Dmin, Dmax, 10)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [5]:
D_train, D_test, U_train, U_test = sklearn.model_selection.train_test_split(D, U,test_size=0.2)

In [6]:
def get_dataset():
    return D_train, D_test, U_train, U_test

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [14]:
def define_model(trial, input_size, output_size):
    # We optimize the number of layers, hidden untis
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = [input_size]
    
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers += [out_features]
    layers += [output_size]
    params_MLP_trial = copy.deepcopy(params_MLP)
    params_MLP_trial['layer_dims'] = layers
    return MLP(params={'solver':None, 'method':params_MLP_trial})

In [15]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1, 1001)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_test, U_train, U_test = get_dataset()

    # Training of the model.
    for epoch in range(100):
        model.train()

        optimizer.zero_grad()
        output = model(D_train)
        loss = loss_fn(output, U_train)
        loss.backward()
        optimizer.step()

        loss_train = loss.item()
        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            output = model(D_test)
            loss = loss_fn(output, U_test)
            
        loss_val = loss.item()

        trial.report(loss_val, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return loss_val

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)


[I 2023-07-26 17:00:00,834] A new study created in memory with name: no-name-8da2133f-d9c3-4e74-b368-91137e37a450


[1, 74, 1001]
MLP(
  (_layers): ModuleList(
    (0): Linear(in_features=1, out_features=74, bias=True)
    (1): Linear(in_features=74, out_features=1001, bias=True)
  )
)
device : cpu
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) to

[I 2023-07-26 17:00:01,063] Trial 0 finished with value: 0.05032679811120033 and parameters: {'n_layers': 1, 'n_units_l0': 74, 'optimizer': 'SGD', 'lr': 0.0005000135895743756}. Best is trial 0 with value: 0.05032679811120033.


torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
[1, 67, 71, 1001]
MLP(
  (_layers): ModuleList(
    (0): Linear(in_features=1, out_features=67, bias=True)
    (1): Linear(in_features=67, out_features=71, bias=True)
    (2): Linear(in_features=71, out_featu

[I 2023-07-26 17:00:01,250] Trial 1 finished with value: 0.007078120484948158 and parameters: {'n_layers': 2, 'n_units_l0': 67, 'n_units_l1': 71, 'optimizer': 'RMSprop', 'lr': 2.3245486589633175e-05}. Best is trial 0 with value: 0.05032679811120033.
[I 2023-07-26 17:00:01,403] Trial 2 finished with value: 0.012184121645987034 and parameters: {'n_layers': 3, 'n_units_l0': 4, 'n_units_l1': 12, 'n_units_l2': 43, 'optimizer': 'SGD', 'lr': 0.006300132847538008}. Best is trial 0 with value: 0.05032679811120033.


...

torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
[1, 114, 1001]
MLP(
  (_layers): ModuleList(
    (0): Linear(in_features=1, out_features=114, bias=True)
    (1): Linear(in_features=114, out_features=1001, bias=True)
  )
)
device : cpu
torch.Size([8, 1001]) torch.Size([8, 1001])
[1, 89, 1001]
MLP(
  (_layers): ModuleList(
    (0): Linear(in_features=1, out_features=89, bias=True)
    (1): Linear(in_features=89, out_features=1001, bias=True)
  )
)
device : cpu
torch.Size([8, 1001]) torch.Size([8, 1001])
[1, 125, 1001]
MLP(
  (_layers): ModuleList(
    (0): Linear(in_features=1, out_features=125, bias=True)
    (1): Linear(in_features=125, out_features=1001, bias=True)
  )
)
device : cpu
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
to

[I 2023-07-26 17:00:13,230] Trial 96 finished with value: 0.06685969978570938 and parameters: {'n_layers': 1, 'n_units_l0': 125, 'optimizer': 'SGD', 'lr': 3.647020541174942e-05}. Best is trial 13 with value: 0.06778555363416672.
[I 2023-07-26 17:00:13,258] Trial 97 pruned. 
[I 2023-07-26 17:00:13,285] Trial 98 pruned. 


torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size

[I 2023-07-26 17:00:13,457] Trial 99 finished with value: 0.067023865878582 and parameters: {'n_layers': 1, 'n_units_l0': 128, 'optimizer': 'SGD', 'lr': 8.999897496466774e-05}. Best is trial 13 with value: 0.06778555363416672.


torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size([8, 1001])
torch.Size([8, 1001]) torch.Size